In [2]:
import psycopg2 # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pandas as pd # type: ignore
from tqdm import tqdm # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import plotly.graph_objects as go # type: ignore
import yfinance as yf # type: ignore
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir("../database")
list_ = os.listdir()

In [24]:
# database: price_fund 
parquet_price_fund = []
for i in list_:
    if "tw_price_fund" in i: 
        parquet_price_fund.append(i)        
df_list = [pd.read_parquet(i) for i in parquet_price_fund]
unpivoted_df = pd.concat(df_list)
# database: maincode_fund
fund_type_code_df = pd.read_csv("tw_maincode_fund__2025.csv")[['code', "fund_type_code"]]

In [ ]:
fund

In [30]:
fund_type_code_df[fund_type_code_df['code'] == int('00968493')]['fund_type_code'].values[0]

'AA1'

In [ ]:
def easy_breakout(df, beck, top_n=1, type_="AE21"):
   
    tdf = df[df['fund_type'] == type_].pivot(values='nav', index='da', columns='code')  
    tdf = tdf.ffill().fillna(0)
    
    pct_change = tdf.pct_change()
    pct_change[(pct_change > 0.1) | (pct_change < -0.1)] = 0 
   
    holdings = []
    cumulative_returns = pd.DataFrame(index=tdf.index)  

   
    for i in range(1, len(tdf)):
        cumulative_returns = cumulative_returns.shift(1).fillna(0) + pct_change
        best_stocks = cumulative_returns.iloc[i].nlargest(top_n).index 
        holdings.append(list(best_stocks)) 
    
    i = 20
    pct_change = pct_change.shift(i)
    pct_change['Best_Holding'] = [None] + holdings
    holds = ""
    holdings_dict = {}
    
    for index, row in pct_change[['Best_Holding']].iterrows():
        if row['Best_Holding'] == None: continue
        current_holdings = set(row['Best_Holding']) 
        if holds != current_holdings:
            holdings_dict[index] = current_holdings 
            holds = current_holdings

    pct_change = pct_change.iloc[2+i:, :]
        
    pct_change['ret'] = pct_change.apply(lambda row: row[row['Best_Holding']].mean(), axis=1)
   
    fig = go.Figure()
   
    fig.add_trace(go.Scatter(
        x=pct_change.index,
        y=pct_change['ret'].cumsum(),
        mode="lines",
        line=dict(color="green"),
        yaxis="y1",
        name="Strategy Return"
    ))

    beck = beck.loc[:].pct_change().cumsum()
    beck.columns = ['pct_change']
    fig.add_trace(go.Scatter(
        x=beck.index,
        y=beck["pct_change"],
        name="Benchmark",
        mode="lines",
        line=dict(color="orange"),
        yaxis="y1"
    ))
    
    fig.update_layout(
        title="Strategy and Benchmark Performance",
        xaxis=dict(title="Date"),
        yaxis=dict(
            title="Strategy Return",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
        ),
        yaxis2=dict(
            title="Benchmark",
            titlefont=dict(color="orange"),
            tickfont=dict(color="orange"),
            overlaying="y", 
            side="right" 
        ),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template="plotly_white"
    )

    fig.show()
    return holdings_dict


config = {
    "host": "10.188.200.16",
    "dbname": "tw",
    "user": "jimmy",
    "password": "123123"
}
conn = psycopg2.connect(**config)
cursor = conn.cursor()

beck = yf.download("0050.TW", start='2020-01-01', progress=False, rounding=2)['Adj Close']
holdings_dict = easy_breakout(df, beck, top_n=3, type_='AA2')
